In [1]:
%load_ext autoreload

%autoreload 2

import datajoint as dj

dj.config["database.host"] = "at-database.ad.bcm.edu"

from neuropixel_pipeline import schemata

Connecting cpapadop@at-database.ad.bcm.edu:3306


## Base

In [2]:
schemata.base.run_populate()
schemata.base.Session()

session_id Session primary key
0


## Probe

In [3]:
schemata.probe.run_populate()
schemata.probe.ProbeType.Electrode()

probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0","shank shank index, starts at 0, advance left to right","shank_col column index, starts at 0, advance left to right","shank_row row index, starts at 0.",x_coord (um) x coordinate of the electrode within the probe.,y_coord (um) y coordinate of the electrode within the probe.
neuropixels 1.0 - 3A,0,0,0,0,16.0,0.0
neuropixels 1.0 - 3A,1,0,1,0,48.0,0.0
neuropixels 1.0 - 3A,2,0,0,1,0.0,20.0
neuropixels 1.0 - 3A,3,0,1,1,32.0,20.0
neuropixels 1.0 - 3A,4,0,0,2,16.0,40.0
neuropixels 1.0 - 3A,5,0,1,2,48.0,40.0
neuropixels 1.0 - 3A,6,0,0,3,0.0,60.0


## Ephys

In [5]:
# schemata.ephys.run_populate()

ephys = schemata.ephys

In [ ]:
ephys.